### Retrieval - Sparse Embedding(BM25)

In [1]:
import json
import os, sys
import datasets
from box import Box

sys.path.append(os.path.dirname(os.path.abspath(os.getcwd())))
from retrieval.sparse.src.sparse_retrieval import SparseRetrieval

In [2]:
config = {"retriever": "../retrieval/sparse/config/sparse_retrieval_config_example.json",
          "reader": "../reader2/config/extraction_config.json"}

config = Box(config)

with open(config.retriever, 'r', encoding='utf-8') as f:
    retriever_config = json.load(f)
    retriever_config = Box(retriever_config)

with open(config.reader, 'r', encoding='utf-8') as f:
    reader_config = json.load(f)
    reader_config = Box(reader_config)

retriever_config, reader_config

(Box({'corpus_data_path': '/data/ephemeral/home/osh/data/v0.0.2/wikipedia_documents.json', 'embedding_method': 'bm25', 'topk': 10, 'tokenizer_model_name': 'klue/bert-base', 'eval_data_path': '/data/ephemeral/home/osh/data/v0.0.2', 'eval_metric': ['hit', 'mrr'], 'retrieval_save_path': '/data/ephemeral/home/osh/level2-mrc-nlp-06/ODQA2/outputs/retrieval/v2-test.csv'}),
 Box({'model_args': {'model_name_or_path': 'uomnf97/klue-roberta-finetuned-korquad-v2', 'config_name': None, 'tokenizer_name': None, 'generation': False, 'retrieval_type': 'dense'}, 'data_args': {'data_path': '/data/ephemeral/home/osh/data/v0.0.4', 'overwrite_cache': True, 'preprocessing_num_workers': None, 'max_seq_length': 384, 'pad_to_max_length': False, 'doc_stride': 128, 'max_answer_length': 90, 'eval_retrieval': True, 'num_clusters': 64, 'top_k_retrieval': 10, 'use_faiss': False}, 'training_args': {'output_dir': '/data/ephemeral/home/osh/level2-mrc-nlp-06/reader2/models/epoch-1', 'evaluation_strategy': 'no', 'eval_ste

In [31]:
import pandas as pd
from transformers import AutoTokenizer

# logger.info("Loading data.")
print("Loading Corpus for Sparse Retrieval")

with open(retriever_config.corpus_data_path, "r", encoding="utf-8") as f:
    wiki = json.load(f)
wiki_df = pd.DataFrame(wiki.values())
wiki_unique_df = wiki_df.drop_duplicates(subset=["text"], keep="first")
contexts = wiki_unique_df["text"].tolist()  # unique text 추출
document_ids = wiki_unique_df["document_id"].tolist()

tokenizer = AutoTokenizer.from_pretrained(retriever_config.tokenizer_model_name, use_fast=True)
retriever = SparseRetrieval(embedding_method=retriever_config.embedding_method,
                            tokenizer=tokenizer,
                            contexts=contexts,
                            document_ids=document_ids)

if retriever_config.embedding_method == "tfidf":
    retriever.get_sparse_embedding_tfidf()
elif retriever_config.embedding_method == "bm25":
    retriever.get_sparse_embedding_bm25()

from datasets import load_from_disk, concatenate_datasets
test_dataset = load_from_disk(retriever_config.eval_data_path)['test']
# org_dataset = load_from_disk(retriever_config.eval_data_path)
# eval_df = concatenate_datasets(
#     [
#         org_dataset["train"].flatten_indices(),
#         org_dataset["validation"].flatten_indices(),
#     ]
# )
# logger.info("Evaluation dataset loaded with %d examples.", len(eval_df))
print("Evaluation dataset loaded with %d examples.", len(test_dataset))

result_df = retriever.retrieve(test_dataset, topk=retriever_config.topk, save=True, retrieval_save_path=retriever_config.retrieval_save_path)

Loading Corpus for Sparse Retrieval
10/21/2024 12:48:53 - INFO - retrieval.sparse.src.sparse_retrieval -    Loading BM25 pickle file.
Evaluation dataset loaded with %d examples. 600
10/21/2024 12:48:56 - INFO - retrieval.sparse.src.sparse_retrieval -    Retrieving for dataset queries.


Calculating BM25 scores:   0%|          | 0/600 [00:00<?, ?it/s]

[query exhaustive search] done in 84.293 s


Sparse retrieval:   0%|          | 0/600 [00:00<?, ?it/s]

10/21/2024 12:50:20 - INFO - retrieval.sparse.src.sparse_retrieval -    Completed retrieval for dataset queries.


### MRC - Extraction Base

In [32]:
result_df.tail(1)

,question,id,document_id
599,구립운석을 이루는 물질 중 널리 알려진 것은?,mrc-0-002605,"[15241, 32440, 52986, 54675, 19751, 42355, 371..."


In [33]:
with open('/data/ephemeral/home/osh/data/v0.0.4/wikipedia_documents.json', 'r', encoding='utf-8') as f:
    wiki = json.load(f)

In [40]:
result_df['context'] = None
for idx, doc_ids in enumerate(result_df['document_id']):
    result_df.loc[idx, 'context'] = ' '.join([wiki[str(i)]['text'] for i in doc_ids])

# result_df.to_csv('./outputs/retrieval/for_reader.csv')
result_df.head()

,question,id,document_id,context
0,유령'은 어느 행성에서 지구로 왔는가?,mrc-1-000653,"[43280, 42242, 47081, 9781, 52764, 26978, 1597...",목성의 대기에서 보이는 줄무늬는 적도와 평행하면서 행성을 둘러싸는 대와 띠라고 불리...
1,용병회사의 경기가 좋아진 것은 무엇이 끝난 이후부터인가?,mrc-1-001113,"[13590, 46223, 20081, 45867, 42721, 31128, 269...",냉전 종식 이후 전 세계적으로 소규모의 끊임없는 국지 분쟁들이 생겨나고 강대국들의 ...
2,돌푸스에게 불특정 기간동안 하원이 잠시 쉬는 것을 건의 받았던 인물은?,mrc-0-002191,"[20554, 42418, 27161, 48970, 20553, 52884, 365...","1933년 3월, 투표 과정의 위법성에 대한 문제제기가 불거졌다. 당시 오스트리아 ..."
3,"마오리언어와 영어, 뉴질랜드 수화를 공식 언어로 사용하는 나라는?",mrc-0-003951,"[25557, 58770, 58774, 4180, 48055, 4248, 48049...","유럽인들의 아메리카와 오세아니아 식민지화로 인해 아메리카와 오세아니아의 문화적, 민..."
4,디엔비엔푸 전투에서 보응우옌잡이 상대한 국가는?,mrc-1-001272,"[18335, 22830, 17274, 40517, 22897, 27405, 485...",1926년 학생 시절 베트남청년혁명당에 가입했고 1930년에 학생 파업을 지지했다는...


In [3]:
import pandas as pd

result_df = pd.read_csv('./outputs/retrieval/for_reader.csv').drop(columns=['Unnamed: 0', 'document_id'])
result_df.head()

,question,id,context
0,유령'은 어느 행성에서 지구로 왔는가?,mrc-1-000653,목성의 대기에서 보이는 줄무늬는 적도와 평행하면서 행성을 둘러싸는 대와 띠라고 불리...
1,용병회사의 경기가 좋아진 것은 무엇이 끝난 이후부터인가?,mrc-1-001113,냉전 종식 이후 전 세계적으로 소규모의 끊임없는 국지 분쟁들이 생겨나고 강대국들의 ...
2,돌푸스에게 불특정 기간동안 하원이 잠시 쉬는 것을 건의 받았던 인물은?,mrc-0-002191,"1933년 3월, 투표 과정의 위법성에 대한 문제제기가 불거졌다. 당시 오스트리아 ..."
3,"마오리언어와 영어, 뉴질랜드 수화를 공식 언어로 사용하는 나라는?",mrc-0-003951,"유럽인들의 아메리카와 오세아니아 식민지화로 인해 아메리카와 오세아니아의 문화적, 민..."
4,디엔비엔푸 전투에서 보응우옌잡이 상대한 국가는?,mrc-1-001272,1926년 학생 시절 베트남청년혁명당에 가입했고 1930년에 학생 파업을 지지했다는...


In [4]:
from extractionQA import ExtractiveQA

reader = ExtractiveQA(config.reader)

/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1545: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [5]:
from datasets import load_from_disk, Dataset

retrieved_test = Dataset.from_pandas(result_df)
retrieved_test

Dataset({
    features: ['question', 'id', 'context'],
    num_rows: 600
})

In [6]:
reader.predict(retrieved_test, './outputs/reader')

Map:   0%|          | 0/600 [00:00<?, ? examples/s]

/data/ephemeral/home/osh/level2-mrc-nlp-06/ODQA2/extractionQA.py:157: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("squad")


10/21/2024 14:19:43 - WARNING - accelerate.utils.other -    Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
10/21/2024 14:19:45 - INFO - extractionQA -    *** Evaluate ***


/opt/conda/lib/python3.10/site-packages/transformers/trainer_pt_utils.py:483: FutureWarning: DistributedTensorGatherer is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


10/21/2024 14:21:45 - INFO - reader2.utils.utils_extraction -    Post-processing 600 example predictions split into 13975 features.


  0%|          | 0/600 [00:00<?, ?it/s]

10/21/2024 14:22:31 - INFO - reader2.utils.utils_extraction -    Saving predictions to ./outputs/reader/predictions.json.
10/21/2024 14:22:31 - INFO - reader2.utils.utils_extraction -    Saving nbest_preds to ./outputs/reader/nbest_predictions.json.
No metric can be presented because there is no correct answer given. Job done!


[{'id': 'mrc-1-000653', 'prediction_text': '사만'},
 {'id': 'mrc-1-001113', 'prediction_text': '냉전'},
 {'id': 'mrc-0-002191', 'prediction_text': '빌헬름 미클라스'},
 {'id': 'mrc-0-003951', 'prediction_text': '뉴질랜드'},
 {'id': 'mrc-1-001272', 'prediction_text': '프랑스'},
 {'id': 'mrc-1-000993', 'prediction_text': '아래턱'},
 {'id': 'mrc-0-005021', 'prediction_text': '대의제도'},
 {'id': 'mrc-1-000163', 'prediction_text': '광독 가스 및 그에 따른 산성비'},
 {'id': 'mrc-0-001283', 'prediction_text': '순조 11년(1811)'},
 {'id': 'mrc-0-004543', 'prediction_text': '고전도성 철'},
 {'id': 'mrc-0-000439', 'prediction_text': '점쟁이'},
 {'id': 'mrc-0-002895', 'prediction_text': '칼라치 전방 약250km 지점'},
 {'id': 'mrc-0-000535', 'prediction_text': '롭 포드'},
 {'id': 'mrc-1-001724', 'prediction_text': '노스햄프턴 교회'},
 {'id': 'mrc-0-000901', 'prediction_text': '아라라기 코요미'},
 {'id': 'mrc-0-001606', 'prediction_text': '석조 궁륭'},
 {'id': 'mrc-0-000266', 'prediction_text': '데르브포르갈'},
 {'id': 'mrc-0-001326', 'prediction_text': '‘자유’지에'},
 {'id': 'mrc-0-0000